In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import folium
import re
import json
# utils.py
from tools import utils as ul

In [2]:
evict = pd.read_csv("data/Eviction_Notices.csv", engine='python', on_bad_lines='skip')

In [3]:
df = evict[['Eviction ID', 'Address', 'Eviction Notice Source Zipcode', 'Neighborhoods - Analysis Boundaries']].dropna()
df = df.rename({'Neighborhoods - Analysis Boundaries': 'Neighborhoods', 'Eviction Notice Source Zipcode': 'Zip'}, axis=1)

In [4]:
df['Zip'] = df['Zip'].astype(np.int64)
counts_df = df[['Zip']].value_counts().reset_index(name='counts')
counts_df_clean = counts_df[counts_df['Zip'] > 90000]

In [5]:
with open('/home/jovyan/sf_geojson/SanFrancisco.Neighborhoods.json', 'r') as jsonFile:
    data = json.load(jsonFile)
tmp = data

# remove ZIP codes not in our dataset
geozips = []
for i in range(len(tmp['features'])):
    if tmp['features'][i]['properties']['id'] in list(df['Zip'].unique()):
        geozips.append(tmp['features'][i])

# creating new JSON object
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips

# save JSON object as updated-file
open("/home/jovyan/sf_geojson/updated-file.json", "w").write(
    json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': '))
)

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/sf_geojson/SanFrancisco.Neighborhoods.json'

In [20]:
def create_map(df, zips, feature, title_desc = ''):
    # df = main table/data frame we read from (pandas DataFrame)
    # zips = column name where ZIP codes are (string)
    # mapped_feature = column name for feature we want to visualize (string)
    # add_text = any additional commentary to be added in the map legend (string)


    # reading of the updated GeoJSON file
    sf_geo = r'/home/jovyan/sf_geojson/updated-file.json'
    # initiating a Folium map with LA's longitude and latitude
    m = folium.Map(location = [37.7749, -122.4194], zoom_start = 11)
    # creating a choropleth map
    m.choropleth(
        geo_data = sf_geo,
        fill_opacity = 0.7,
        line_opacity = 0.2,
        data = df,
        # refers to which key within the GeoJSON to map the ZIP code to
        key_on = 'feature.properties.id',
        # first element contains location information, second element contains feature of interest
        columns = [zips, feature],
        fill_color = 'RdYlGn',
        legend_name = (' ').join(feature.split('_')).title() + ' ' + title_desc + ' Across SF'
    )
    folium.LayerControl().add_to(m)
    # save map with filename based on the feature of interest
    m.save(outfile = feature + '_map.html')

In [19]:
create_map(counts_df_clean, zips = 'Zip', feature = 'counts')

/srv/conda/envs/notebook/lib/python3.10/site-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
